- 지영님 코드 (내가 시도해본 건 잘 안됨)

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import json
import re

# ✅ 브라우저 설정
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 10)

# ✅ 아고다 호텔 목록 페이지 (제주도)
driver.get("https://www.agoda.com/ko-kr/search?city=16901")
time.sleep(20)

hotel_links = set()
scroll_pause = 2
same_count = 0
max_same = 3
last_height = 0

print("🚀 스크롤 시작")

# ✅ 전체 호텔 데이터 저장 리스트
all_hotels = []

# ✅ 호텔 링크 추출 함수 (수정됨)
def extract_hotel_links():
    links = set()
    # 기존 'a[data-selenium="hotel-name"]' 대신, 호텔 상세 링크는 a 태그의 클래스 "PropertyCard__Link"에 있음
    hotel_elements = driver.find_elements(By.CSS_SELECTOR, 'a.PropertyCard__Link')
    if hotel_elements:
        print("✅ 구조 감지: [a.PropertyCard__Link] 사용")
    else:
        # 만약 없으면 다른 셀렉터 시도
        hotel_elements = driver.find_elements(By.CSS_SELECTOR, 'a[data-testid="item-name"]')
        if hotel_elements:
            print("✅ 구조 감지: [data-testid='item-name'] 사용")
        else:
            print("❌ 호텔 링크 요소를 찾지 못했습니다.")
            return links

    for el in hotel_elements:
        link = el.get_attribute("href")
        if link and "hotel" in link:
            links.add(link)
    return links

# ✅ 스크롤 + 링크 수집 루프
while True:
    try:
        more_button = driver.find_element(By.CSS_SELECTOR, 'button[data-selenium="load-more-button"]')
        if more_button.is_displayed():
            print("➡️ '더 보기' 버튼 클릭")
            driver.execute_script("arguments[0].click();", more_button)
            time.sleep(3)
    except:
        pass

    current_height = driver.execute_script("return document.body.scrollHeight")
    for i in range(last_height, current_height, 300):
        driver.execute_script(f"window.scrollTo(0, {i});")
        time.sleep(1.2)
    last_height = current_height

    new_links = extract_hotel_links()
    new_only = new_links - hotel_links
    before = len(hotel_links)

    if new_only:
        print(f"\n🌐 새로 수집된 호텔 링크 {len(new_only)}개:")
        for l in new_only:
            print(f" - {l}")

    for link in new_only:
        hotel_links.add(link)
        if len(hotel_links) >= 5:  # 테스트용으로 5개 수집 후 중단
            print("✅ 테스트용: 5개 링크 수집 완료. 중단합니다.")
            break

    after = len(hotel_links)
    print(f"🔎 현재 누적 수집 호텔 수: {after}")

    if len(hotel_links) >= 5:
        break

    if after == before:
        same_count += 1
        print(f"⚠️ 변화 없음 감지: {same_count}/{max_same}")
        if same_count >= max_same:
            print("✅ 더 이상 로딩되지 않음. 종료.")
            break
    else:
        same_count = 0

# ✅ 최종 수집 결과 출력
print(f"\n🏨 최종 수집된 호텔 링크 수: {len(hotel_links)}")
for link in hotel_links:
    print(link)

# ✅ 호텔 상세 정보 및 리뷰 수집 함수
def crawl_hotel_details(link):
    print(f"\n🔗 호텔 상세 페이지 진입: {link}")
    driver.get(link)
    time.sleep(3)

    hotel_info = {}

    try:
        hotel_name = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1[data-selenium="hotel-header-name"]'))).text
        print(f"\n✅ 호텔명: {hotel_name}")
    except:
        hotel_name = "호텔명 없음"
        print("❌ 호텔명 수집 실패")
    hotel_info["hotel_name"] = hotel_name

    try:
        paragraphs = driver.find_elements(By.XPATH, '//p[contains(@class, "Typographystyled__TypographyStyled-sc-j18mtu-0")]')
        hotel_desc = "\n".join([p.text.strip() for p in paragraphs if len(p.text.strip()) > 50])
        print(f"\n🏨 호텔 설명 요약:\n{hotel_desc}")
    except:
        hotel_desc = ""
        print("❌ 호텔 설명 수집 실패")
    hotel_info["description"] = hotel_desc

    try:
        rating_candidates = driver.find_elements(By.XPATH, '//p[contains(@class, "Typographystyled__TypographyStyled-sc-j18mtu-0")]')
        for el in rating_candidates:
            text = el.text.strip()
            if re.match(r'^\d+(\.\d+)?$', text):
                hotel_info["rating"] = text
                break
    except:
        hotel_info["rating"] = ""
    print(f"\n⭐️ 호텔 평점: {hotel_info.get('rating', '없음')}")

    # ✅ 객실 정보 수집
    rooms = []
    try:
        time.sleep(2)
        room_blocks = driver.find_elements(By.CSS_SELECTOR, 'div.MasterRoom')
        for room in room_blocks:
            try:
                room_name = room.find_element(By.CSS_SELECTOR, '[data-selenium="masterroom-title-name"]').text
                try:
                    image_tag = room.find_element(By.CSS_SELECTOR, '[data-selenium="MasterRoom-infoPhoto-image"] img')
                    image_url = image_tag.get_attribute("src")
                    if image_url and image_url.startswith("//"):
                        image_url = "https:" + image_url
                except:
                    image_url = "대표 이미지 없음"
                try:
                    price = room.find_element(By.CSS_SELECTOR, '[data-element-name="final-price"]').text.replace('\n', '').strip()
                except:
                    price = "가격 정보 없음"
                amenities = []
                try:
                    amenity_tags = room.find_elements(By.CSS_SELECTOR, 'ul.MasterRoom-amenities li .MasterRoom-amenitiesTitle')
                    for a in amenity_tags:
                        amenities.append(a.text.strip())
                except:
                    pass
                rooms.append({"room_name": room_name, "image": image_url, "price": price, "amenities": amenities})
            except:
                continue
    except:
        print("❌ 객실 정보 수집 실패")
    hotel_info["rooms"] = rooms

    # ✅ 편의시설 수집
    facilities_list = []
    try:
        facility_section = driver.find_element(By.XPATH, '//h2[contains(text(), "편의 시설/서비스")]')
        driver.execute_script("arguments[0].scrollIntoView(true);", facility_section)
        time.sleep(2)
        facilities = driver.find_elements(By.CSS_SELECTOR, 'div[data-selenium^="facilities-section"] ul li')
        for item in facilities:
            text = item.text.strip()
            if text:
                facilities_list.append(text)
    except:
        print("❌ 편의시설 수집 실패")
    hotel_info["facilities"] = facilities_list

    # ✅ 리뷰 수집 (2페이지까지만)
    reviews_list = []
    page = 1
    max_pages = 2
    while page <= max_pages:
        print(f"\n🔄 페이지 {page} 리뷰 수집 중...")
        try:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'ol.Review-comments')))
            reviews = driver.find_elements(By.CSS_SELECTOR, 'ol.Review-comments > li')
            for review in reviews:
                try:
                    title = review.find_element(By.CSS_SELECTOR, '[data-testid="review-title"]').text
                    content = review.find_element(By.CSS_SELECTOR, '[data-testid="review-comment"]').text
                    reviewer_text = review.find_element(By.CSS_SELECTOR, '[data-info-type="reviewer-name"]').text.strip()
                    if "(" in reviewer_text:
                        nickname, country = reviewer_text.split("(")[0].strip(), reviewer_text.split("(")[1].replace(")", "").strip()
                    else:
                        nickname, country = reviewer_text, "정보 없음"
                    group_type = review.find_element(By.CSS_SELECTOR, '[data-info-type="group-name"]').text
                    room_type = review.find_element(By.CSS_SELECTOR, '[data-info-type="room-type"]').text
                    stay_info = review.find_element(By.CSS_SELECTOR, '[data-info-type="stay-detail"]').text
                    date = review.find_element(By.CSS_SELECTOR, '.Review-statusBar-left').text.replace("작성일: ", "")
                    reviews_list.append({
                        "title": title,
                        "content": content,
                        "nickname": nickname,
                        "country": country,
                        "group_type": group_type,
                        "room_type": room_type,
                        "stay_info": stay_info,
                        "date": date
                    })
                except:
                    continue
            next_button = driver.find_elements(By.XPATH, f'//button[@aria-label="이용후기 페이지로 이동하기 {page + 1}"]')
            if next_button and page < max_pages:
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button[0])
                time.sleep(1)
                driver.execute_script("arguments[0].click();", next_button[0])
                time.sleep(3)
                page += 1
            else:
                break
        except:
            break

    hotel_info["reviews"] = reviews_list
    all_hotels.append(hotel_info)

# ✅ 호텔 상세 페이지 순회
for link in hotel_links:
    crawl_hotel_details(link)

# ✅ JSON 저장
with open("C:\\Users\\JaneNahm\\Documents\\2025-1학기\\데이터엔지니어링\\프로젝트1\\agoda_hotel_details.json", "w", encoding="utf-8") as f:
    json.dump(all_hotels, f, ensure_ascii=False, indent=2)

print("\n✅ 모든 호텔 정보가 'agoda_hotel_details.json' 파일에 저장되었습니다.")
driver.quit()


🚀 스크롤 시작
✅ 구조 감지: [a.PropertyCard__Link] 사용

🌐 새로 수집된 호텔 링크 49개:
 - https://www.agoda.com/ko-kr/jeju-dongyang-condo/hotel/jeju-island-kr.html?countryId=212&finalPriceView=1&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2025-04-11&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=KRW&isFreeOccSearch=false&los=1&searchrequestid=f59445a6-bea3-4284-bc7b-8911e218b2a7
 - https://www.agoda.com/ko-kr/h31611941/hotel/jeju-island-kr.html?countryId=212&finalPriceView=1&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2025-04-11&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=KRW&isFreeOccSearch=false&tspTypes=7,1,16,-1&los=1&searchrequestid=f59445a6-bea3-4284-bc7b-8911e218b2a7
 - https://www

1. **특정 날짜를 선택**
2. **페이지를 넘기면서 크롤링하고 싶어**
3. **모든 호텔의 모든 리뷰를 크롤링**
4. **호텔의 이미지 url도 같이 저장**

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import json
import re

# 1. 브라우저 설정 및 드라이버 초기화
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 15)

# 2. 사용자 설정: 날짜, 도시, 총 페이지 수 (예: 37)
check_in = "2025-07-01"
check_out = "2025-07-04"
city = "16901"  # 제주도
total_pages = 5   # 여기서는 테스트로 5페이지만 크롤링 (전체 37페이지 가능)
base_url = "https://www.agoda.com/ko-kr/search"

# 3. 전체 호텔 데이터 저장 리스트
all_hotel_links = set()

def get_page_url(page):
    # 페이지 URL 구성 (필요에 따라 추가 파라미터 조정)
    return (f"{base_url}?city={city}&checkIn={check_in}&checkOut={check_out}"
            f"&adults=2&rooms=1&locale=ko-kr&page={page}")

# 4. 각 페이지에서 호텔 링크 수집 함수
def collect_hotel_links_from_page(url):
    driver.get(url)
    time.sleep(20)  # 페이지 로딩 대기
    # 스크롤로 호텔 카드 로딩 (여러 번)
    for _ in range(5):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(10)
    # 수집: 상세 링크는 'a.PropertyCard__Link' 로 추출 (절대 URL)
    links = set()
    hotel_elements = driver.find_elements(By.CSS_SELECTOR, 'a.PropertyCard__Link')
    for el in hotel_elements:
        link = el.get_attribute("href")
        if link and "hotel" in link:
            links.add(link)
    return links

# 5. 페이지별 호텔 링크 수집 (테스트로 total_pages 만큼)
print("🚀 페이지별 호텔 링크 수집 시작")
for page in range(1, total_pages+1):
    page_url = get_page_url(page)
    print(f"\n📄 수집 중: 페이지 {page} URL: {page_url}")
    new_links = collect_hotel_links_from_page(page_url)
    print(f"   페이지 {page}에서 {len(new_links)}개 링크 수집됨")
    all_hotel_links.update(new_links)
    time.sleep(5)

print(f"\n🏨 최종 수집된 호텔 링크 수: {len(all_hotel_links)}")
for link in all_hotel_links:
    print(link)

# 6. 호텔 상세 페이지 크롤링 함수 (호텔명, 이미지, 평점, 리뷰 등)
all_hotels_details = []

def crawl_hotel_details(link):
    print(f"\n🔗 호텔 상세 페이지 진입: {link}")
    driver.get(link)
    time.sleep(5)
    hotel_info = {}
    try:
        hotel_name = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1[data-selenium="hotel-header-name"]'))).text.strip()
    except:
        try:
            # 만약 h1 요소가 없으면 h3 요소로 시도
            hotel_name = driver.find_element(By.CSS_SELECTOR, 'h3[data-selenium="hotel-name"]').text.strip()
        except:
            hotel_name = "호텔명 없음"
    hotel_info["hotel_name"] = hotel_name
    print(f"✅ 호텔명: {hotel_name}")

    # 대표 이미지 (호텔 상세 페이지 상단의 이미지)
    try:
        image = driver.find_element(By.CSS_SELECTOR, 'div.Overlay button img').get_attribute("src")
        if image.startswith("//"):
            image = "https:" + image
    except:
        image = ""
    hotel_info["image"] = image

    # 평점 및 리뷰 수 (호텔 상세 페이지의 리뷰 영역)
    try:
        rating_element = driver.find_element(By.CSS_SELECTOR, 'span.sc-jrAGrp.sc-kEjbxe.fzPhrN.ehWyCi')
        rating = rating_element.text.strip()
    except:
        rating = ""
    hotel_info["rating"] = rating

    try:
        review_text_element = driver.find_element(By.CSS_SELECTOR, 'p.sc-jrAGrp.sc-kEjbxe.iDzlXT.bAPNf')
        review_text = review_text_element.text.strip()
        review_count = review_text.split()[0]
    except:
        review_count = ""
    hotel_info["review_count"] = review_count

    # 리뷰 수집: 모든 리뷰 페이지 순회
    reviews_list = []
    page = 1
    while True:
        print(f"   🔄 리뷰 페이지 {page} 수집 중...")
        try:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'ol.Review-comments')))
            review_elements = driver.find_elements(By.CSS_SELECTOR, 'ol.Review-comments > li')
            for review in review_elements:
                try:
                    title = review.find_element(By.CSS_SELECTOR, '[data-testid="review-title"]').text.strip()
                except:
                    title = ""
                try:
                    content = review.find_element(By.CSS_SELECTOR, '[data-testid="review-comment"]').text.strip()
                except:
                    content = ""
                try:
                    reviewer = review.find_element(By.CSS_SELECTOR, '[data-info-type="reviewer-name"]').text.strip()
                except:
                    reviewer = ""
                reviews_list.append({
                    "title": title,
                    "content": content,
                    "reviewer": reviewer
                })
        except Exception as e:
            print("   ❌ 리뷰 수집 오류:", e)
            break

        # 다음 리뷰 페이지 버튼: aria-label 형식으로 "이용후기 페이지로 이동하기 {page+1}"
        try:
            next_buttons = driver.find_elements(By.XPATH, f'//button[contains(@aria-label, "이용후기 페이지로 이동하기 {page + 1}")]')
            if next_buttons:
                driver.execute_script("arguments[0].scrollIntoView(true);", next_buttons[0])
                time.sleep(1)
                next_buttons[0].click()
                time.sleep(5)
                page += 1
            else:
                break
        except:
            break
    hotel_info["reviews"] = reviews_list

    # 기타 호텔 정보(설명, 편의시설 등)도 추가 가능하지만 여기서는 생략
    all_hotels_details.append(hotel_info)

# 7. 수집된 호텔 링크에 대해 상세 페이지 크롤링
print("\n🚀 호텔 상세 페이지 크롤링 시작")
for idx, link in enumerate(all_hotel_links):
    print(f"\n========== 호텔 {idx+1} 크롤링 ==========")
    crawl_hotel_details(link)
    # 너무 많은 호텔을 크롤링하면 부담되므로 테스트 후 딜레이 추가
    time.sleep(3)

# 8. 최종 JSON 저장
with open("agoda_hotel_details.json", "w", encoding="utf-8") as f:
    json.dump(all_hotels_details, f, ensure_ascii=False, indent=2)

print("\n✅ 모든 호텔 정보가 'agoda_hotel_details.json' 파일에 저장되었습니다.")

driver.quit()


🚀 페이지별 호텔 링크 수집 시작

📄 수집 중: 페이지 1 URL: https://www.agoda.com/ko-kr/search?city=16901&checkIn=2025-07-01&checkOut=2025-07-04&adults=2&rooms=1&locale=ko-kr&page=1
   페이지 1에서 12개 링크 수집됨

📄 수집 중: 페이지 2 URL: https://www.agoda.com/ko-kr/search?city=16901&checkIn=2025-07-01&checkOut=2025-07-04&adults=2&rooms=1&locale=ko-kr&page=2
   페이지 2에서 12개 링크 수집됨

📄 수집 중: 페이지 3 URL: https://www.agoda.com/ko-kr/search?city=16901&checkIn=2025-07-01&checkOut=2025-07-04&adults=2&rooms=1&locale=ko-kr&page=3
   페이지 3에서 98개 링크 수집됨

📄 수집 중: 페이지 4 URL: https://www.agoda.com/ko-kr/search?city=16901&checkIn=2025-07-01&checkOut=2025-07-04&adults=2&rooms=1&locale=ko-kr&page=4
   페이지 4에서 12개 링크 수집됨

📄 수집 중: 페이지 5 URL: https://www.agoda.com/ko-kr/search?city=16901&checkIn=2025-07-01&checkOut=2025-07-04&adults=2&rooms=1&locale=ko-kr&page=5
   페이지 5에서 98개 링크 수집됨

🏨 최종 수집된 호텔 링크 수: 232
https://www.agoda.com/ko-kr/the-best-seongsan/hotel/jeju-island-kr.html?countryId=212&finalPriceView=1&isShowMobileAppPrice=false&cid=-1&n

KeyboardInterrupt: 

In [5]:
all_hotels_details

[{'hotel_name': '더 베스트 제주 성산 (The Best Jeju Seongsan)',
  'image': '',
  'rating': '',
  'review_count': '',
  'reviews': [{'title': '“가성비가 좋은 호텔 ”',
    'content': '지하주차장보단 야외주차장을 이용하세요. 지하는 칸이 좁아서 주차는 가능하나 하차가 불가능합니다... 1층에 편의점 있어서 이용하기 좋았고, 2박할 때 요청하면 문앞에 바구니에 2일차의 물과 수건을 가져다 주는게 좋았습니다. 다만 좀 오래된 듯한 기물(화장실 거울, 세면대)의 상태와 방의 입구가 좁은데, 문이 열리는 방향으로 신발장이 있어서 신발을 신고 벗기가 불편했습니다. 욕실 문은 불투명하고 틈으로 안에 사람이 좀 보여서 친한관계가 아니라면 민망한 상황이 발생할 수 있구요. 방안에서 욕실안에 샤워장이 불투명하게 보이는 구조입니다. (블라인드가 있으나 양쪽 틈이 좀 보임) 침대는 바스락바스락 하고 베개도 높이가 적당해서 좋았어요. 가구들도 무난무난하고, 방에서 성산이 한눈에 들어오는게 정말 좋았습니다. 저희 방에선 앞에 있는 유채꽃 밭도 보여서 뷰는 정말 훌륭했어요. 조식은 이용하지 않았고, 아쉬운 점은 다른방에서 물을 사용하면 그 소음이 조금 크다는 것과 개인적으로 비 선호하는 화장실의 인테리어 그리고 비누!가 없었습니다. 핸드워시랑 바니워시만 있고 비누는 없더라구요. 미리 챙기시면 좋겠습니다.',
    'reviewer': 'Yu (대한민국)'},
   {'title': '“뷰가 너무 좋고 직원들이 너무 친절한 호텔.”',
    'content': '당일로 급하게 잡은 숙소인대 체크인부터 프론트에 있는 조철진지배인님의 좋은 서비스를 받아 너무 기분 좋게 시작하였네요!방에 들어가니 객실 상태도 너무 깨끗했궁 뷰도 좋았어요.위치도 너무 좋구 직원분들이 너무 친절하여 재방문 하고 싶은 호텔입니다: )',
    'reviewer': '정은 (대한민국)'},
   {'

1. **호텔의 전체 평점, 호텔의 이미지 url, 각 리뷰별 평점, 총 리뷰수 제대로 나오게 수정**
2. **크롤링 범위 줄임**




### 1 
> 이미지 url이 수집되지 않고<br>모든 호텔링크의 리뷰를 가져오고 있어서 시간이 오래걸림

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import json
import re

# ============================
# 1. 브라우저 설정 및 드라이버 초기화
# ============================
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 30)

# ============================
# 2. 사용자 설정 (날짜, 도시, 페이지 수)
# ============================
check_in = "2025-07-01"
check_out = "2025-07-04"
city = "16901"  # 제주도
total_pages = 2  # 호텔 링크는 2페이지까지만 수집
base_url = "https://www.agoda.com/ko-kr/search"

# ============================
# 3. 호텔 링크 수집 함수
# ============================
all_hotel_links = set()

def get_page_url(page):
    return (f"{base_url}?city={city}&checkIn={check_in}&checkOut={check_out}"
            f"&adults=2&rooms=1&locale=ko-kr&page={page}")

def collect_hotel_links_from_page(url):
    driver.get(url)
    # 명시적으로 호텔 카드가 로딩될 때까지 대기
    try:
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li[data-selenium="hotel-item"]')))
    except Exception as e:
        print("호텔 카드 로딩 대기 실패:", e)
    time.sleep(10)
    # 스크롤로 추가 로딩 (여러 번 스크롤)
    for _ in range(5):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
    links = set()
    # 호텔 카드 내부의 a 태그 (상세 링크)
    hotel_elements = driver.find_elements(By.CSS_SELECTOR, 'li[data-selenium="hotel-item"]')
    for hotel in hotel_elements:
        try:
            link = hotel.find_element(By.CSS_SELECTOR, 'a.PropertyCard__Link').get_attribute("href")
            if link and "hotel" in link:
                links.add(link)
        except Exception as e:
            continue
    return links

print("🚀 페이지별 호텔 링크 수집 시작")
for page in range(1, total_pages+1):
    page_url = get_page_url(page)
    print(f"\n📄 수집 중: 페이지 {page} URL: {page_url}")
    new_links = collect_hotel_links_from_page(page_url)
    print(f"   페이지 {page}에서 {len(new_links)}개 링크 수집됨")
    all_hotel_links.update(new_links)
    time.sleep(5)

print(f"\n🏨 최종 수집된 호텔 링크 수: {len(all_hotel_links)}")
for link in all_hotel_links:
    print(link)

# ============================
# 4. 호텔 상세 페이지 및 리뷰 크롤링 함수
# ============================
all_hotels_details = []

def crawl_hotel_details(link):
    print(f"\n🔗 호텔 상세 페이지 진입: {link}")
    driver.get(link)
    time.sleep(5)
    hotel_info = {}

    # 호텔명 추출: h1[data-selenium="hotel-header-name"] 또는 h3[data-selenium="hotel-name"]
    try:
        hotel_name = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1[data-selenium="hotel-header-name"]'))).text.strip()
    except:
        try:
            hotel_name = driver.find_element(By.CSS_SELECTOR, 'h3[data-selenium="hotel-name"]').text.strip()
        except:
            hotel_name = "호텔명 없음"
    hotel_info["hotel_name"] = hotel_name
    print(f"✅ 호텔명: {hotel_name}")

    # 대표 이미지 추출 (상세 페이지 상단 이미지)
    try:
        image = driver.find_element(By.CSS_SELECTOR, 'div.Overlay button img').get_attribute("src")
        if image.startswith("//"):
            image = "https:" + image
    except:
        image = ""
    hotel_info["image"] = image

    # 평점 추출
    try:
        rating = driver.find_element(By.CSS_SELECTOR, 'span.sc-jrAGrp.sc-kEjbxe.fzPhrN.ehWyCi').text.strip()
    except:
        rating = ""
    hotel_info["rating"] = rating

    # 리뷰 수 추출
    try:
        review_text = driver.find_element(By.CSS_SELECTOR, 'p.sc-jrAGrp.sc-kEjbxe.iDzlXT.bAPNf').text.strip()
        review_count = review_text.split()[0]
    except:
        review_count = ""
    hotel_info["review_count"] = review_count

    # 리뷰 수집: 리뷰 페이지 1과 2에서 각각 최대 5개씩 수집
    reviews_list = []
    review_page = 1
    max_review_pages = 2
    while True:
        print(f"   🔄 리뷰 페이지 {review_page} 수집 중...")
        try:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'ol.Review-comments')))
        except Exception as e:
            print("   리뷰 영역 로딩 실패:", e)
            break

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        review_items = soup.select('ol.Review-comments > li')
        if not review_items:
            print("   리뷰 요소 없음, 종료.")
            break

        # 각 페이지에서 최대 5개 리뷰만 수집
        for item in review_items[:5]:
            # 평점
            rating_elem = item.select_one('div.Review-comment-leftScore')
            review_rating = rating_elem.get_text(strip=True) if rating_elem else ""
            # 평점 텍스트
            rating_text_elem = item.select_one('div.Review-comment-leftScoreText')
            review_rating_text = rating_text_elem.get_text(strip=True) if rating_text_elem else ""
            # 제목
            title_elem = item.select_one('h4[data-testid="review-title"]')
            review_title = title_elem.get_text(strip=True) if title_elem else ""
            # 내용
            comment_elem = item.select_one('p[data-testid="review-comment"]')
            review_comment = comment_elem.get_text(strip=True) if comment_elem else ""
            # 리뷰어 이름과 국가
            reviewer_elem = item.select_one('div[data-info-type="reviewer-name"]')
            if reviewer_elem:
                name_tag = reviewer_elem.find('strong')
                reviewer_name = name_tag.get_text(strip=True) if name_tag else ""
                span_tags = reviewer_elem.find_all('span')
                reviewer_country = span_tags[1].get_text(strip=True) if len(span_tags) >= 2 else ""
            else:
                reviewer_name = ""
                reviewer_country = ""
            # 작성일
            date_elem = item.select_one('div.Review-statusBar-left span')
            review_date = date_elem.get_text(strip=True).replace("작성일:", "").strip() if date_elem else ""
            
            reviews_list.append({
                "review_rating": review_rating,
                "review_rating_text": review_rating_text,
                "title": review_title,
                "comment": review_comment,
                "reviewer": reviewer_name,
                "country": reviewer_country,
                "review_date": review_date
            })
        
        if review_page >= max_review_pages:
            print(f"✅ 최대 {max_review_pages}페이지까지 수집 완료.")
            break

        try:
            # 다음 리뷰 페이지 버튼: aria-label에 "이용후기 페이지로 이동하기 {review_page+1}" 포함
            next_buttons = driver.find_elements(By.XPATH, f'//button[contains(@aria-label, "이용후기 페이지로 이동하기 {review_page + 1}")]')
            if next_buttons:
                driver.execute_script("arguments[0].scrollIntoView(true);", next_buttons[0])
                time.sleep(1)
                next_buttons[0].click()
                time.sleep(5)
                review_page += 1
            else:
                print("   다음 리뷰 페이지 버튼 없음, 리뷰 수집 종료.")
                break
        except Exception as e:
            print("   다음 페이지 이동 오류:", e)
            break

    hotel_info["reviews"] = reviews_list
    all_hotels_details.append(hotel_info)

# ============================
# 5. 호텔 상세 페이지 순회
# ============================
print("\n🚀 호텔 상세 페이지 크롤링 시작")
for idx, link in enumerate(all_hotel_links):
    print(f"\n========== 호텔 {idx+1} 크롤링 ==========")
    crawl_hotel_details(link)
    time.sleep(3)

# ============================
# 6. 최종 결과 JSON 저장
# ============================
with open("agoda_hotel_details.json", "w", encoding="utf-8") as f:
    json.dump(all_hotels_details, f, ensure_ascii=False, indent=2)

print("\n✅ 모든 호텔 정보가 'agoda_hotel_details.json' 파일에 저장되었습니다.")
driver.quit()


🚀 페이지별 호텔 링크 수집 시작

📄 수집 중: 페이지 1 URL: https://www.agoda.com/ko-kr/search?city=16901&checkIn=2025-07-01&checkOut=2025-07-04&adults=2&rooms=1&locale=ko-kr&page=1
   페이지 1에서 98개 링크 수집됨

📄 수집 중: 페이지 2 URL: https://www.agoda.com/ko-kr/search?city=16901&checkIn=2025-07-01&checkOut=2025-07-04&adults=2&rooms=1&locale=ko-kr&page=2
   페이지 2에서 98개 링크 수집됨

🏨 최종 수집된 호텔 링크 수: 196
https://www.agoda.com/ko-kr/shilla-stay-plus-ihotewoo/hotel/jeju-kr.html?countryId=212&finalPriceView=1&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&adults=2&children=0&rooms=1&maxRooms=0&checkIn=2025-07-1&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=1&showReviewSubmissionEntry=false&currencyCode=KRW&isFreeOccSearch=false&tspTypes=-1&los=3&searchrequestid=65ab022d-19c3-4a09-baf3-8478f078cb37
https://www.agoda.com/ko-kr/grabel-hotel/hotel/jeju-island-kr.html?countryId=212&finalPriceView=1&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&a

KeyboardInterrupt: 

In [10]:
all_hotels_details

[{'hotel_name': '신라스테이 플러스 이호테우 오션 프론트 (Shilla Stay Plus Iho Tewoo Ocean Front)',
  'image': '',
  'rating': '9.0',
  'review_count': '1,744',
  'reviews': [{'review_rating': '10.0',
    'review_rating_text': '최고',
    'title': '“힐링여행”',
    'comment': '오랫만에 엄마 모시고 동생들이랑 제주여행을 왔습니다\n신라스테이에 4인이 머물수 있는 객실이 있어서 좋았는데 온돌이 있어서 더 좋았습니다\n온돌을 선호하시는 엄마도 매우 만족하셨습니다\n이호테우 신라스테이 특유의 건축 디자인 때문에 커튼을 열면 건너편 객실이 보여서 옷갈아 입거나할때 신경써야 하는 부분도 있었지만 건너편 유리에 비치는 바닷가 모습은 또 다른 낭만적인 모습이었습니다\n쌀쌀하고 바람부는 날이었지만 온수 야외수영장에서 물놀이도 즐겼습니다\n엄마도 정말 즐거워하셔서 이곳을 택하길 잘했다는 생각이 들었습니다\n커피숍에서 바라보는 바다의 모습도 정말 훌륭합니다  바다를 보면서 몇시간도 멍때리고 있을수 있을것 같습니다\n엄마와 동생들과 커피마시며  이런저런 이야기를 하는것 자체가 힐링이었습니다\n마음껏 웃고 좋은 풍경도 보고 맛있는 음식도 먹고 정말 최고의 여행이었습니다\n주차안내 하시는 분과 프론트 직원분들 또 객실을 정비해 주시던 직원분도 참 친절하게 대해주시고 깔끔하게 정리해주셔서 우리의 여행이 더욱더 즐거워졌습니다\n다음에 제주에 갈 기회가 된다면 또다시 꼭 머물고 싶은 곳입니다\n덕분에 잘 쉬었다 갑니다\n감사합니다',
    'reviewer': 'MIKYUNG',
    'country': '대한민국',
    'review_date': '2025년 2월 18일'},
   {'review_rating': '10.0',
    'review_rating_text': '최고',
    'title': 

### 2
> 이미지 url 개선 <br> 총 10개 호텔의 각 리뷰 10개씩만 크롤링

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import json
import re

# ============================
# 1. 브라우저 설정 및 드라이버 초기화
# ============================
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 30)

# ============================
# 2. 사용자 설정 (날짜, 도시, 페이지 수)
# ============================
check_in = "2025-07-01"
check_out = "2025-07-04"
city = "16901"  # 제주도
total_pages = 2  # 호텔 링크는 1~2페이지까지만 수집 (테스트용)
base_url = "https://www.agoda.com/ko-kr/search"

# ============================
# 3. 호텔 링크 수집 함수 (느린 스크롤 방식)
# ============================
all_hotel_links = set()

def get_page_url(page):
    return (f"{base_url}?city={city}&checkIn={check_in}&checkOut={check_out}"
            f"&adults=2&rooms=1&locale=ko-kr&page={page}")

def collect_hotel_links_from_page(url):
    driver.get(url)
    time.sleep(20)  # 페이지 로딩 대기
    # 느린 스크롤: 300px씩 스크롤하며 대기
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        for pos in range(0, last_height, 300):
            driver.execute_script("window.scrollTo(0, arguments[0]);", pos)
            time.sleep(3)
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    links = set()
    hotel_elements = driver.find_elements(By.CSS_SELECTOR, 'li[data-selenium="hotel-item"]')
    for hotel in hotel_elements:
        try:
            link = hotel.find_element(By.CSS_SELECTOR, 'a.PropertyCard__Link').get_attribute("href")
            if link and "hotel" in link:
                links.add(link)
        except Exception as e:
            continue
    return links

print("🚀 페이지별 호텔 링크 수집 시작")
for page in range(1, total_pages+1):
    page_url = get_page_url(page)
    print(f"\n📄 수집 중: 페이지 {page} URL: {page_url}")
    new_links = collect_hotel_links_from_page(page_url)
    print(f"   페이지 {page}에서 {len(new_links)}개 링크 수집됨")
    all_hotel_links.update(new_links)
    time.sleep(5)

print(f"\n🏨 최종 수집된 호텔 링크 수: {len(all_hotel_links)}")
for link in all_hotel_links:
    print(link)

# ============================
# 4. 호텔 상세 페이지 및 리뷰 크롤링 함수
# ============================
all_hotels_details = []

def crawl_hotel_details(link):
    print(f"\n🔗 호텔 상세 페이지 진입: {link}")
    driver.get(link)
    time.sleep(5)
    hotel_info = {}

    # 호텔명 추출
    try:
        hotel_name = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1[data-selenium="hotel-header-name"]'))).text.strip()
    except:
        try:
            hotel_name = driver.find_element(By.CSS_SELECTOR, 'h3[data-selenium="hotel-name"]').text.strip()
        except:
            hotel_name = "호텔명 없음"
    hotel_info["hotel_name"] = hotel_name
    print(f"✅ 호텔명: {hotel_name}")

    # 대표 이미지 추출 (수정됨)
    try:
        # 제공된 HTML 구조를 기반으로 이미지 추출
        image = driver.find_element(By.CSS_SELECTOR, 'img.sc-bkzZxe.sc-idOhPF.gBpyeJ').get_attribute("src")
        if image.startswith("//"):
            image = "https:" + image
    except Exception as e:
        image = ""
        print("이미지 추출 오류:", e)
    hotel_info["image"] = image

    # 평점 추출
    try:
        rating = driver.find_element(By.CSS_SELECTOR, 'span.sc-jrAGrp.sc-kEjbxe.fzPhrN.ehWyCi').text.strip()
    except:
        rating = ""
    hotel_info["rating"] = rating

    # 리뷰 수 추출
    try:
        review_text = driver.find_element(By.CSS_SELECTOR, 'p.sc-jrAGrp.sc-kEjbxe.iDzlXT.bAPNf').text.strip()
        review_count = review_text.split()[0]
    except:
        review_count = ""
    hotel_info["review_count"] = review_count

    # 리뷰 수집: 각 호텔의 리뷰는 최대 2페이지에서, 각 페이지마다 최대 5개 리뷰만 수집
    reviews_list = []
    review_page = 1
    max_review_pages = 2
    while True:
        print(f"   🔄 리뷰 페이지 {review_page} 수집 중...")
        try:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'ol.Review-comments')))
        except Exception as e:
            print("   리뷰 영역 로딩 실패:", e)
            break
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        review_items = soup.select('ol.Review-comments > li')
        if not review_items:
            print("   리뷰 요소 없음, 종료.")
            break

        # 각 페이지에서 최대 5개 리뷰만 추출
        for item in review_items[:5]:
            rating_elem = item.select_one('div.Review-comment-leftScore')
            review_rating = rating_elem.get_text(strip=True) if rating_elem else ""
            rating_text_elem = item.select_one('div.Review-comment-leftScoreText')
            review_rating_text = rating_text_elem.get_text(strip=True) if rating_text_elem else ""
            title_elem = item.select_one('h4[data-testid="review-title"]')
            review_title = title_elem.get_text(strip=True) if title_elem else ""
            comment_elem = item.select_one('p[data-testid="review-comment"]')
            review_comment = comment_elem.get_text(strip=True) if comment_elem else ""
            reviewer_elem = item.select_one('div[data-info-type="reviewer-name"]')
            if reviewer_elem:
                name_tag = reviewer_elem.find('strong')
                reviewer_name = name_tag.get_text(strip=True) if name_tag else ""
                span_tags = reviewer_elem.find_all('span')
                reviewer_country = span_tags[1].get_text(strip=True) if len(span_tags) >= 2 else ""
            else:
                reviewer_name = ""
                reviewer_country = ""
            date_elem = item.select_one('div.Review-statusBar-left span')
            review_date = date_elem.get_text(strip=True).replace("작성일:", "").strip() if date_elem else ""
            
            reviews_list.append({
                "review_rating": review_rating,
                "review_rating_text": review_rating_text,
                "title": review_title,
                "comment": review_comment,
                "reviewer": reviewer_name,
                "country": reviewer_country,
                "review_date": review_date
            })
        
        if review_page >= max_review_pages:
            print(f"✅ 최대 {max_review_pages}페이지까지 수집 완료.")
            break

        try:
            next_buttons = driver.find_elements(By.XPATH, f'//button[contains(@aria-label, "이용후기 페이지로 이동하기 {review_page + 1}")]')
            if next_buttons:
                driver.execute_script("arguments[0].scrollIntoView(true);", next_buttons[0])
                time.sleep(2)
                next_buttons[0].click()
                time.sleep(5)
                review_page += 1
            else:
                print("   다음 리뷰 페이지 버튼 없음, 리뷰 수집 종료.")
                break
        except Exception as e:
            print("   다음 페이지 이동 오류:", e)
            break
    hotel_info["reviews"] = reviews_list
    all_hotels_details.append(hotel_info)

# ============================
# 5. 호텔 상세 페이지 순회 (최대 10개 호텔만 크롤링)
# ============================
print("\n🚀 호텔 상세 페이지 크롤링 시작")
for idx, link in enumerate(list(all_hotel_links)[:10]):
    print(f"\n========== 호텔 {idx+1} 크롤링 ==========")
    crawl_hotel_details(link)
    time.sleep(3)

# ============================
# 6. 최종 결과 JSON 저장
# ============================
with open("agoda_hotel_details.json", "w", encoding="utf-8") as f:
    json.dump(all_hotels_details, f, ensure_ascii=False, indent=2)

print("\n✅ 모든 호텔 정보가 'agoda_hotel_details.json' 파일에 저장되었습니다.")
driver.quit()


🚀 페이지별 호텔 링크 수집 시작

📄 수집 중: 페이지 1 URL: https://www.agoda.com/ko-kr/search?city=16901&checkIn=2025-07-01&checkOut=2025-07-04&adults=2&rooms=1&locale=ko-kr&page=1


KeyboardInterrupt: 